In [2]:
import matplotlib.pyplot as plt
plt.rcParams['image.interpolation'] = 'none'
import numpy as np
import cv2
import math
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from scipy.ndimage import gaussian_filter1d
import itertools

In [3]:
def display_images(imgs, titles, width=5):
    n = len(imgs)
    rows = math.ceil(n / 3)
    cols = n // rows
    h, w = imgs[0].shape[:2]
    fig, axs = plt.subplots(rows, cols, figsize=(width * cols, width * h / w * rows))
    axs = axs.flatten()
    for i in range(rows * cols):
        if i < n:
            img = imgs[i]
            axs[i].imshow(img, cmap='gray' if img.ndim == 2 else None)
            axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


def to_float(img):
    img = img.astype(np.float64)
    return (img - np.min(img)) / (np.max(img) - np.min(img))

def to_uint8(img):
    norm = (img - np.min(img)) / (np.max(img) - np.min(img))
    return np.round(norm * 255).astype(np.uint8)

In [ ]:
def noise_filter(src, option='median', med_k=5, gauss_sigma=1.5, bilat_d=6, bilat_sigmaColor=75, bilat_sigmaSpace=75, morph_kernel=5):
    if option == 'median':
        return cv2.medianBlur(src, med_k)
    elif option == 'gaussian':
        ksize = round(6 * gauss_sigma + 1)
        ksize += 1 - (ksize % 2)
        return cv2.GaussianBlur(src, (ksize, ksize), gauss_sigma)
    elif option == 'bilateral':
        return cv2.bilateralFilter(src, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)
    elif option == 'morph_max':
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (morph_kernel, morph_kernel))
        return cv2.morphologyEx(src, cv2.MORPH_DILATE, kernel)
    else:
        raise ValueError(f"Invalid filter option: {option}")

In [ ]:
def find_threshold(src, option='mixture'):
    if option == 'otsu':
        _, thr = cv2.threshold(src, 0, 256, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thr
    elif option == 'mixture':
        hist, bins = np.histogram(src, bins=256, range=(0, 256))
        sample = cv2.resize(src, (14, 14), interpolation=cv2.INTER_CUBIC).reshape(-1, 1)
        gmm = GaussianMixture(n_components=2, covariance_type='full')
        gmm.fit(sample)
        means = gmm.means_.flatten()
        m1, m2 = np.sort(means)
        hist_smooth = gaussian_filter1d(hist, sigma=2)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        mask = (bin_centers > m1) & (bin_centers < m2)
        thr = bin_centers[mask][np.argmin(hist_smooth[mask])]
        return thr
    else:
        raise ValueError(f"Invalid thresholding option: {option}")

In [ ]:
def get_edges(src, option="laplacian", canny_thr1=50, canny_thr2=150):
    if option == "canny":
        return cv2.Canny(src, canny_thr1, canny_thr2, apertureSize=3)
    elif option == "laplacian":
        src_blur = cv2.GaussianBlur(src, (5,5), 0)
        lap = cv2.Laplacian(src_blur, ddepth=cv2.CV_64F)
        z1 = np.roll(lap, 1, axis=0) * np.roll(lap, -1, axis=0)
        z2 = np.roll(lap, 1, axis=1) * np.roll(lap, -1, axis=1)
        return np.logical_or(z1 < 0, z2 < 0).astype(np.uint8) * 255
    else:
        raise ValueError(f"Invalid edges option: {option}")

In [ ]:
def get_intersection(l1, l2):
    x1, y1, x2, y2 = l1
    x3, y3, x4, y4 = l2
    A1, B1, C1 = y2 - y1, x1 - x2, x2*y1 - x1*y2
    A2, B2, C2 = y4 - y3, x3 - x4, x4*y3 - x3*y4
    det = A1 * B2 - A2 * B1
    if det == 0:
        return None
    x = (B1 * C2 - B2 * C1) / det
    y = (C1 * A2 - C2 * A1) / det
    return (x, y)

def point_line_distance(x0, y0, x1, y1, x2, y2):
    num = abs((y2 - y1)*x0 - (x2 - x1)*y0 + x2*y1 - y2*x1)
    den = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return num / den

In [ ]:
def get_filtered_lines(edges, high_thr=240, low_thr=150, centroids_sep=100, line_dist_thr=30):
    few_raw = cv2.HoughLines(edges, 1, np.pi/180, threshold=high_thr)
    raw = cv2.HoughLines(edges, 1, np.pi/180, threshold=low_thr)
    if few_raw is None or raw is None:
        return []
    few_lines = few_raw[:, 0, :]
    lines = raw[:, 0, :]

    intersections = []
    for l1, l2 in itertools.combinations(few_lines, 2):
        pt = get_intersection(l1, l2)
        if pt is not None and np.all(np.isfinite(pt)):
            intersections.append(pt)
    if not intersections:
        return few_lines
    intersections = np.array(intersections)

    clustering = DBSCAN(eps=40, min_samples=5).fit(intersections)
    labels = clustering.labels_
    if np.sum(labels != -1) == 0:
        return few_lines

    unique_labels = np.unique(labels[labels != -1])
    cluster_sizes = np.array([np.sum(labels == k) for k in unique_labels])
    centroids = np.stack([intersections[labels == k].mean(axis=0) for k in unique_labels])
    centroids = centroids[np.argsort(cluster_sizes)[::-1]]

    points = []
    for centroid in centroids:
        if all(centroid[0]-p[0] > centroids_sep and centroid[1]-p[1] > centroids_sep for p in points):
            points.append(centroid)
    points = np.array(points)

    filtered_lines = []
    for rho, theta in lines:
        for x, y in points:
            if point_line_distance(x, y, rho, theta) < line_dist_thr:
                filtered_lines.append([rho, theta])
                break
    filtered_lines = np.array(filtered_lines)

    return filtered_lines

def remove_duplicate_lines(lines, shape):
    if len(lines) == 0:
        return np.array([])

    lines = np.array(lines)
    h, w = shape[:2]
    d = np.hypot(h, w)
    theta_mean = np.mean(lines[:, 1])
    keep = np.ones(len(lines), dtype=bool)

    def get_line_points(rho, theta):
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = np.array([x0 + d * -b, y0 + d * a])
        pt2 = np.array([x0 - d * -b, y0 - d * a])
        return pt1, pt2

    def intersection(p1, p2, q1, q2):
        A = np.array([[p2[0] - p1[0], q1[0] - q2[0]],
                      [p2[1] - p1[1], q1[1] - q2[1]]])
        b = np.array([q1[0] - p1[0], q1[1] - p1[1]])
        if np.linalg.matrix_rank(A) < 2:
            return None
        t = np.linalg.solve(A, b)
        return p1 + t[0] * (p2 - p1)

    for i in range(len(lines)):
        if not keep[i]:
            continue
        rho1, theta1 = lines[i]
        p1, p2 = get_line_points(rho1, theta1)
        for j in range(i + 1, len(lines)):
            if not keep[j]:
                continue
            rho2, theta2 = lines[j]
            q1, q2 = get_line_points(rho2, theta2)
            inter = intersection(p1, p2, q1, q2)
            if inter is not None:
                x, y = inter
                if 0 <= x < w and 0 <= y < h:
                    dist_i = abs(theta1 - theta_mean)
                    dist_j = abs(theta2 - theta_mean)
                    if dist_i > dist_j:
                        keep[i] = False
                        break
                    else:
                        keep[j] = False

    return lines[keep]